In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Read in Constructor Results CSV

In [2]:
csv_file = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructorResults.csv"
constructorResults_df = pd.read_csv(csv_file)
constructorResults_df.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,NaN
1,2,18,2,8.0,NaN
2,3,18,3,9.0,NaN
3,4,18,4,5.0,NaN
4,5,18,5,2.0,NaN


##### see if any values in status and drop if nothing

In [3]:
constructorResults_df['status'].value_counts()

D    17
Name: status, dtype: int64

#### Transform premise Dataframe

### Read in Constructors.csv

In [4]:
csv_file2 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructors.csv"
constructor_df = pd.read_csv(csv_file2)
constructor_df.head()

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,NaN
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,NaN
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_F1,NaN
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,NaN


##### see if any values in unnamed: 5 and drop if nothing

In [5]:
constructor_df['Unnamed: 5'].unique()

array([nan])

##### drop unnamed: 5 because only nan values

### Read in Constructor Standings.csv

In [6]:
csv_file3 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructorStandings.csv"
constructorstanding_df = pd.read_csv(csv_file3)
constructorstanding_df

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,Unnamed: 7
0,1,18,1,14.0,1,1,1,NaN
1,2,18,2,8.0,3,3,0,NaN
2,3,18,3,9.0,2,2,0,NaN
3,4,18,4,5.0,4,4,0,NaN
4,5,18,5,2.0,5,5,0,NaN
...,...,...,...,...,...,...,...,...
11891,26928,988,5,53.0,7,7,0,NaN
11892,26929,988,4,57.0,6,6,0,NaN
11893,26930,988,15,5.0,10,10,0,NaN
11894,26931,988,1,30.0,9,9,0,NaN


##### see if any values in unnamed: 7 and drop if nothing

In [7]:
constructorstanding_df['Unnamed: 7'].unique()

array([nan])

##### drop unnamed: 7 because only nan values

### Read in races.csv

In [8]:
csv_file4 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/races.csv"
races_df = pd.read_csv(csv_file4)
races_df.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


#### connect to local databases

In [9]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgresql

#### check for tables

In [ ]:
engine.table_names()

In [ ]:
constructorResults_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
constructor_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
constructorstanding_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)